In [1]:
# pip install opencv-python

In [2]:
import matplotlib
import sklearn
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import pathlib
import os
import cv2
import datetime

from os import listdir
from os.path import isfile, isdir, join
from pathlib import Path

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from keras.models import load_model
from keras.models import Model
from tensorflow.keras.layers import concatenate

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

print("Versions of key libraries")
print("---")
print("tensorflow: ", tf.__version__)
print("numpy:      ", np.__version__)
print("matplotlib: ", matplotlib.__version__)
print("sklearn:    ", sklearn.__version__)
print(tf.config.list_physical_devices())

Versions of key libraries
---
tensorflow:  2.9.0
numpy:       1.22.3
matplotlib:  3.5.2
sklearn:     1.1.1
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
members_param = [
    {
        "model_name": "general",
        "file_path": "./models/medium_best.hdf5",
        "class_names": ['arowana', 'betta', 'goldfish', 'flowerhorn'],
        "img_width": 224,
        "img_height":160
    },
    {
        "model_name": "arowana",
        "file_path": "./models/arowana_best.hdf5",
        "class_names": ['arowana', 'not arowana'],
        "img_width": 128,
        "img_height":128
    },
    {
        "model_name": "betta",
        "file_path": "./models/betta_best.hdf5",
        "class_names": ['betta', 'not betta'],
        "img_width": 128,
        "img_height":128
    },
    {
        "model_name": "goldfish",
        "file_path": "./models/goldfish_best.hdf5",
        "class_names": ['goldfish', 'not goldfish'],
        "img_width": 128,
        "img_height":128
    },
    {
        "model_name": "flowerhorn",
        "file_path": "./models/luohan_best.hdf5",
        "class_names": ['flowerhorn', 'not flowerhorn'],
        "img_width": 128,
        "img_height":128
    }
]

modelname   = 'arbitrator'
BATCH_SIZE = 32 # Big enough to measure an F1-score
AUTOTUNE = tf.data.AUTOTUNE # Adapt preprocessing and prefetching dynamically
SHUFFLE_BUFFER_SIZE = 1024 # Shuffle the training data by a chunck of 1024 observations
IMG_WIDTH = 224
IMG_HEIGHT = 160
CHANNELS = 3
SEED = 7
N_LABELS = 4
EPOCHS = 50
OPT_IDX = 1
ACTIVATION = 'softmax'

class_names = ''
modelname = modelname+"_"+ACTIVATION+"_"+str(EPOCHS)+"_"+str(BATCH_SIZE)+"_"+str(OPT_IDX)
optmzs = ['adam', optimizers.RMSprop(learning_rate=0.0001)]
optmz = optmzs[OPT_IDX]

In [4]:
def readImagesFromDir(base_img_path='dataset/'):
    dirs = [d for d in listdir(base_img_path) if isdir(join(base_img_path, d)) and not d.startswith('.') and not d in ['oranda', 'common_goldfish']]

    print(dirs)

    X, y = [], []

    for d in dirs:
        img_path = base_img_path + d + "/"
        files = [f for f in listdir(img_path) if isfile(join(img_path, f))]
        X = X + [os.path.join(img_path, f) for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        y = y + [d for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        #print(d)

    data_dir = Path(base_img_path)
    image_count = len(list(data_dir.glob('*/*.*')))
    
    return X, y


def preprocess_image(filename, label):
    """Function that returns a tuple of normalized image array and labels array.
    Args:
        filename: string representing path to image
        label: 0/1 one-dimensional array of size N_LABELS
    """
    # Read an image from a file
    images = {}
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=CHANNELS)
#     image_resized = tf.image.resize_with_pad(image_decoded, IMG_HEIGHT, IMG_WIDTH, antialias=False)
#     image_normalized = image_resized / 255.0
    for i,m in enumerate(members_param):
        image_resized = tf.image.resize_with_pad(image_decoded, m["img_height"], m["img_width"], antialias=False)
        image_normalized = image_resized / 255.0
        images["input_"+str(i)] = image_normalized
        
    return images, label


def create_dataset(filenames, labels, is_training=True):
    """Load and parse dataset.
    Args:
        filenames: list of image paths
        labels: numpy array of shape (BATCH_SIZE, N_LABELS)
        is_training: boolean to indicate training mode
    """
    
#     imageDS = 
    
#     labelDS = tf.data.Dataset.from_tensor_slices(labels)
    # Create a first dataset of file paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    # Parse and preprocess observations in parallel
    dataset = dataset.map(preprocess_image, num_parallel_calls=AUTOTUNE)
    
    if is_training == True:
        #dataset = dataset.take(BATCH_SIZE)
        # This is a small dataset, only load it once, and keep it in memory.
        #dataset = dataset.cache()
        #dataset = dataset.repeat()
        # Shuffle the data each buffer size
        dataset = dataset.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
        #dataset = dataset.repeat()
        
    # Batch the data for multiple steps    
    dataset = dataset.batch(BATCH_SIZE)
    # Fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset
    
def visualize_image(original, augmented):
    org_img = tf.keras.utils.array_to_img(original)
    fig = plt.figure()
    plt.subplot(1,2,1)
    plt.title('Original image')
    plt.imshow(org_img)

    aug_img = tf.keras.utils.array_to_img(augmented)
    plt.subplot(1,2,2)
    plt.title('Augmented image')
    plt.imshow(aug_img)


def predict_new_image(img_file):    
    #img = tf.keras.utils.load_img(
    #    img_file, target_size=(IMG_HEIGHT, IMG_WIDTH), keep_aspect_ratio=True
    #)
    img = tf.keras.utils.load_img(
        img_file, target_size=None, keep_aspect_ratio=True
    )

    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.keras.preprocessing.image.smart_resize(img_array, size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    image_normalized = img_array / 255.0  # tf.image.per_image_standardization(img_array)

    saved_model = load_model(model_file)

    predictions = saved_model.predict(image_normalized)
    score = tf.nn.softmax(predictions[0])
    #score = tf.sigmoid(predictions[0])
    #score = tf.nn.sigmoid_cross_entropy_with_logits(labels=class_names, logits=predictions[0])
    #score = tf.math.sigmoid(predictions[0])
    #score = tf.tanh(predictions[0])

    #model.evaluate(img_array)
    plt.imshow(img)
    plt.show()

    #print(predictions[0])

    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )
    print()

In [5]:
X, Y = readImagesFromDir()

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=SEED)

lb = LabelBinarizer()
lb.fit(y_train)
class_names = lb.classes_
# Loop over all labels and show them    
N_LABELS = len(class_names)
for (i, label) in enumerate(class_names):
    print("{}. {}".format(i, label))

# transform the targets of the training and test sets
y_train_bin = lb.transform(y_train)
y_val_bin = lb.transform(y_val)

print(y_train_bin.shape)
print(y_val_bin.shape)

['arowana', 'betta', 'goldfish', 'luohan']
0. arowana
1. betta
2. goldfish
3. luohan
(9166, 4)
(2292, 4)


In [6]:
# Print examples of fish and their binary targets
for i in range(10):
    print(X_train[len(X_train)-1 - i], y_train_bin[len(y_train_bin)-1 - i])

dataset/luohan/Flowerhorn10380.jpg [0 0 0 1]
dataset/arowana/240_F_73601716_vduxsAMnoi1s681yzgQQa7WmvEUNeFuQ.jpg [1 0 0 0]
dataset/luohan/Flowerhorn11710.jpg [0 0 0 1]
dataset/betta/BettaFish11692.jpg [0 1 0 0]
dataset/betta/BettaFish10300.jpg [0 1 0 0]
dataset/arowana/Arowana_100303.jpg [1 0 0 0]
dataset/arowana/Arowana_100756.jpg [1 0 0 0]
dataset/luohan/Red-Texas-Flowerhorn.jpg [0 0 0 1]
dataset/goldfish/Goldfish2517.jpg [0 0 1 0]
dataset/arowana/Arowana_101967.jpg [1 0 0 0]


In [7]:
# train_ds = []
# val_ds = []

train_ds = create_dataset(X_train, y_train_bin)
val_ds = create_dataset(X_val, y_val_bin, is_training=False)

# for images, labels in train_ds.take(1):
#     print("Shape of features array:", images.numpy().shape)
#     print("Shape of labels array:", labels.numpy().shape)
#     #plt.imshow(f.numpy().astype("uint8"))
#     for i in range(5):
#         ax = plt.subplot(2, 3, i + 1)
#         img = tf.keras.utils.array_to_img(images[i])
#         plt.imshow(img)
#         plt.title(class_names[np.argmax(labels[i])])
#         plt.axis("off")

In [8]:
def add_prefix(model, prefix: str, i:str, custom_objects=None):
    config = model.get_config()
    new_to_old = {}
    for layer in config['layers']:
#         print(layer)
        new_name = prefix + i + layer['config']['name']
        if layer['class_name']=='InputLayer':
            new_name = "input_"+i
        new_to_old[new_name] = layer['config']['name']
#         layer['name'] = new_name
        layer['config']['name'] = new_name
    if config['name'].startswith('sequential'):
        new_model = tf.keras.Sequential().from_config(config, custom_objects)
    else:
        new_model = tf.keras.Model().from_config(config, custom_objects)
    for layer in new_model.layers:
        layer.set_weights(model.get_layer(new_to_old[layer.name]).get_weights())
    
    return new_model

# load models from file
def load_all_models(members_param):
    all_models = list()
    prefix = "ensemble_"
    
    for i, m in enumerate(members_param):
        # define filename for this ensemble
        filename = m["file_path"]
        # load model from file
        model = load_model(filename)
        # add to list of members
        all_models.append(add_prefix(model,prefix,str(i)))
        print('>loaded %s' % filename)
    return all_models
 
# define stacked model from multiple member input models
def define_stacked_model(members):
    # update all layers in all models to not be trainable
    for i in range(len(members)):
        model = members[i]
        for layer in model.layers:
            # make not trainable
            layer.trainable = False
    # define multi-headed input
    ensemble_visible = [model.input for model in members]
    # concatenate merge output from each model
    ensemble_outputs = [model.output for model in members]
    merge = concatenate(ensemble_outputs)
    hidden = Dense(12, activation='relu')(merge)
    output = Dense(N_LABELS, activation=ACTIVATION)(hidden)
    model = Model(inputs=ensemble_visible, outputs=output)

    # compile
    model.compile(loss='categorical_crossentropy', optimizer=optmz, metrics=['accuracy'])
    return model
member_models = load_all_models(members_param)
model = define_stacked_model(member_models)
model.summary()

>loaded ./models/medium_best.hdf5
>loaded ./models/arowana_best.hdf5
>loaded ./models/betta_best.hdf5
>loaded ./models/goldfish_best.hdf5
>loaded ./models/luohan_best.hdf5
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_0 (InputLayer)           [(None, 160, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 ensemble_0conv2d_12 (Conv2D)   (None, 160, 224, 32  2432        ['input_0[0][0]']                
                                )                                                                 
                                                                                                  
 ensemble_0max_poolin

 ensemble_4max_pooling2d (MaxPo  (None, 64, 64, 32)  0           ['ensemble_4conv2d[0][0]']       
 oling2D)                                                                                         
                                                                                                  
 ensemble_0conv2d_17 (Conv2D)   (None, 5, 7, 768)    3539712     ['ensemble_0max_pooling2d_16[0][0
                                                                 ]']                              
                                                                                                  
 ensemble_1dropout (Dropout)    (None, 64, 64, 32)   0           ['ensemble_1max_pooling2d[0][0]']
                                                                                                  
 ensemble_2dropout (Dropout)    (None, 64, 64, 32)   0           ['ensemble_2max_pooling2d[0][0]']
                                                                                                  
 ensemble_

 ensemble_2dropout_2 (Dropout)  (None, 16, 16, 8)    0           ['ensemble_2max_pooling2d_2[0][0]
                                                                 ']                               
                                                                                                  
 ensemble_3dropout_2 (Dropout)  (None, 16, 16, 8)    0           ['ensemble_3max_pooling2d_2[0][0]
                                                                 ']                               
                                                                                                  
 ensemble_4dropout_2 (Dropout)  (None, 16, 16, 8)    0           ['ensemble_4max_pooling2d_2[0][0]
                                                                 ']                               
                                                                                                  
 ensemble_0dense_17 (Dense)     (None, 64)           16448       ['ensemble_0dense_16[0][0]']     
          

In [9]:
                                                                                # Step 1
modelname       = modelname+"_"+str(datetime.datetime.now())[:-7].replace(' ','_').replace(":",'-')
folderpath      = 'models/'
model_json      = folderpath + modelname + ".json"
with open(model_json, "w") as json_file:
    json_file.write(model.to_json())
model_file      = folderpath + modelname + ".hdf5"
checkpoint      = ModelCheckpoint(model_file, 
                                  monitor='val_accuracy', 
                                  verbose=0, 
                                  save_best_only=True, 
                                  mode='max')

csv_logger      = CSVLogger(folderpath+modelname +'.csv')                       # Step 2
callbacks_list  = [checkpoint,csv_logger]                                       # Step 3

print("Path to model:", model_file)
print("Path to log:  ", folderpath+modelname+'.csv')

Path to model: models/arbitrator_softmax_50_32_1_2022-10-17_00-21-39.hdf5
Path to log:   models/arbitrator_softmax_50_32_1_2022-10-17_00-21-39.csv


In [10]:
import time as time

start = time.time()
history = model.fit(train_ds,                            # Training data and label
          validation_data=val_ds,   # Validation data and label
          epochs=EPOCHS,                       # The amount of epochs to be trained
          batch_size=BATCH_SIZE,                   
          shuffle=True,                     # To shuffle the training data
          callbacks=callbacks_list)         # Callbacks to execute the checkpoints

end = time.time()
duration = round(((end - start)/60), 2)
print("duration = ", duration, " minutes")

Epoch 1/50
287/287 [==============================] - 57s 165ms/step - loss: 2.0807 - accuracy: 0.3299 - val_loss: 2.0100 - val_accuracy: 0.4660
Epoch 2/50
287/287 [==============================] - 47s 160ms/step - loss: 1.9515 - accuracy: 0.5112 - val_loss: 1.8898 - val_accuracy: 0.6078
Epoch 3/50
287/287 [==============================] - 47s 157ms/step - loss: 1.8300 - accuracy: 0.6717 - val_loss: 1.7744 - val_accuracy: 0.6745
Epoch 4/50
287/287 [==============================] - 47s 160ms/step - loss: 1.7127 - accuracy: 0.6968 - val_loss: 1.6589 - val_accuracy: 0.6950
Epoch 5/50
287/287 [==============================] - 49s 166ms/step - loss: 1.6019 - accuracy: 0.7087 - val_loss: 1.5515 - val_accuracy: 0.7308
Epoch 6/50
287/287 [==============================] - 46s 158ms/step - loss: 1.5003 - accuracy: 0.7599 - val_loss: 1.4539 - val_accuracy: 0.8421
Epoch 7/50
287/287 [==============================] - 49s 168ms/step - loss: 1.4099 - accuracy: 0.8599 - val_loss: 1.3645 - val_ac

NotFoundError: Graph execution error:

2 root error(s) found.
  (0) NOT_FOUND:  NewRandomAccessFile failed to Create/Open: dataset/luohan/227_1500355650.jpg : The system cannot find the file specified.
; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]]
	 [[IteratorGetNext/_10]]
  (1) NOT_FOUND:  NewRandomAccessFile failed to Create/Open: dataset/luohan/227_1500355650.jpg : The system cannot find the file specified.
; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_9353]

In [ ]:
plotpath  = folderpath+modelname+'_plot.png'
plot_model(model, 
           to_file=plotpath, 
           show_shapes=True, 
           show_layer_names=False,
           rankdir='TB')
print("Path to plot:", plotpath)

In [ ]:
                                                                                # Step 1
modelGo = load_model(model_file)

predicts    = modelGo.predict(val_ds)                                            # Step 2
print("Prediction completes.")

In [ ]:
                                                                                # Step 1
                                                                                # Step 2
predout     = np.argmax(predicts,axis=1)
testout     = np.argmax(y_val_bin,axis=1)

testScores  = metrics.accuracy_score(testout,predout)                           # Step 3

                                                                                # Step 4
print("Best accuracy (on testing dataset): %.2f%%" % (testScores*100))
print(metrics.classification_report(testout,
                                    predout,
                                    target_names=class_names,
                                    digits=4))

report = metrics.classification_report(testout,
                                    predout,
                                    target_names=class_names,
                                    digits=4,
                                      output_dict=True)

df = pd.DataFrame(report).transpose()
df.to_csv(folderpath+modelname+'_report.csv')


In [ ]:
resdf = pd.DataFrame()
resdf["file"] = X_val
resdf["fish"] = y_val
resdf["testout"] = testout
resdf["predout"] = predout
print(resdf)

In [ ]:
#print all wrong predition
resdf[resdf.testout !=resdf.predout]